In [69]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Загрузка данных
df = pd.read_csv('diamonds_train.csv')
test_df = pd.read_csv('diamonds_test.csv')

# Сохраняем id для submission
test_ids = test_df['id']

print("Размер train данных:", df.shape)
print("Размер test данных:", test_df.shape)
df.head()

Размер train данных: (43018, 10)
Размер test данных: (5379, 10)


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.51,Good,D,SI2,63.9,55.0,1180,5.04,5.10,3.24
1,0.72,Ideal,E,VS2,60.8,57.0,3091,5.79,5.82,3.53
2,0.70,Very Good,D,VVS2,62.8,60.0,4022,5.65,5.69,3.56
3,0.36,Ideal,D,SI1,61.2,57.0,663,4.59,4.63,2.82
4,0.54,Very Good,D,SI1,60.0,59.8,1593,5.30,5.34,3.18


In [70]:
def feature_engineering(df):
    df = df.copy()

    # Удаляем id если есть
    if 'id' in df.columns:
        df = df.drop('id', axis=1)

    # === ВСЕГО 3 НОВЫХ ПРИЗНАКА ===
    df['volume'] = df['x'] * df['y'] * df['z']
    df['table_depth_ratio'] = df['table'] / (df['depth'] + 1e-6)
    df['avg_dimension'] = (df['x'] + df['y'] + df['z']) / 3

    return df

# Применяем feature engineering
df_processed = feature_engineering(df)
test_processed = feature_engineering(test_df)

print("Колонки после feature engineering:")
print(df_processed.columns.tolist())

Колонки после feature engineering:
['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z', 'volume', 'table_depth_ratio', 'avg_dimension']


In [71]:
cut_mapping = {'Fair': 1, 'Good': 2, 'Very Good': 3, 'Premium': 4, 'Ideal': 5}
color_mapping = {'J': 1, 'I': 2, 'H': 3, 'G': 4, 'F': 5, 'E': 6, 'D': 7}
clarity_mapping = {'I1': 1, 'SI2': 2, 'SI1': 3, 'VS2': 4, 'VS1': 5, 'VVS2': 6, 'VVS1': 7, 'IF': 8}

# Применяем кодирование
df_processed['cut'] = df_processed['cut'].map(cut_mapping)
df_processed['color'] = df_processed['color'].map(color_mapping)
df_processed['clarity'] = df_processed['clarity'].map(clarity_mapping)

test_processed['cut'] = test_processed['cut'].map(cut_mapping)
test_processed['color'] = test_processed['color'].map(color_mapping)
test_processed['clarity'] = test_processed['clarity'].map(clarity_mapping)

print("После кодирования категориальных переменных:")
print(df_processed.head())

После кодирования категориальных переменных:
   carat  cut  color  clarity  depth  table  price     x     y     z  \
0   0.51    2      7        2   63.9   55.0   1180  5.04  5.10  3.24   
1   0.72    5      6        4   60.8   57.0   3091  5.79  5.82  3.53   
2   0.70    3      7        6   62.8   60.0   4022  5.65  5.69  3.56   
3   0.36    5      7        3   61.2   57.0    663  4.59  4.63  2.82   
4   0.54    3      7        3   60.0   59.8   1593  5.30  5.34  3.18   

       volume  table_depth_ratio  avg_dimension  
0   83.280960           0.860720       4.460000  
1  118.953234           0.937500       5.046667  
2  114.448660           0.955414       4.966667  
3   59.929794           0.931373       4.013333  
4   90.000360           0.996667       4.606667  


In [72]:
# Разделяем на features и target
X = df_processed.drop('price', axis=1)
y = df_processed['price']
X_test = test_processed

# Заполняем пропуски
X = X.fillna(X.median())
X_test = X_test.fillna(X.median())

print(f"Финальная форма данных: X {X.shape}, X_test {X_test.shape}")
print(f"Колонки: {list(X.columns)}")

# Разделяем на тренировочную и валидационную выборку
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Train: {X_train.shape}, Val: {X_val.shape}")

Финальная форма данных: X (43018, 12), X_test (5379, 12)
Колонки: ['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z', 'volume', 'table_depth_ratio', 'avg_dimension']
Train: (34414, 12), Val: (8604, 12)


In [73]:

print("\n=== Быстрый RandomForest ===")
fast_rf = RandomForestRegressor(
    n_estimators=100,      # Меньше деревьев
    max_depth=15,          # Ограничиваем глубину
    min_samples_split=10,  # Ускоряем обучение
    n_jobs=-1,            # Параллельное обучение
    random_state=42
)

fast_rf.fit(X, y)
rf_pred = fast_rf.predict(X_test)
rf_rmse = np.sqrt(mean_squared_error(y, fast_rf.predict(X)))
print(f"RMSE на train: {rf_rmse:.2f}")



=== Быстрый RandomForest ===
RMSE на train: 348.60


In [74]:
# Предсказываем на тестовых данных
predictions = fast_rf.predict(X_test)

# Создаем submission файл
submission_df = pd.DataFrame({
    'id': test_ids,
    'price': predictions
})

# Сохраняем в CSV файл
submission_df.to_csv('submission.csv', index=False)

print(submission_df.head())
print(f"\nСтатистика предсказаний:")
print(f"Min цена: {predictions.min():.2f}")
print(f"Max цена: {predictions.max():.2f}")
print(f"Mean цена: {predictions.mean():.2f}")

   id        price
0   0  4082.077433
1   1  2465.035802
2   2   748.769540
3   3  1163.923991
4   4   863.155776

Статистика предсказаний:
Min цена: 370.33
Max цена: 17981.67
Mean цена: 3933.11
